In [12]:
!pip install optuna lightgbm scikit-learn pandas numpy


In [13]:
# Importar librerías
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import optuna

# -------------------------
# 1. Carga de datos
# -------------------------
df = pd.read_csv('../data/processed/booking_data.csv')



C:\Users\Alumno_AI\AppData\Local\Temp\ipykernel_7468\2982348590.py:16: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/processed/booking_data.csv')


In [14]:
# -----------------------------
# 2. FEATURE ENGINEERING
# -----------------------------
df['review_sentiment_bin'] = (df['review_sentiment'] == 'positive').astype(int)

# Asegurar mes y trimestre
df['fecha_llegada'] = pd.to_datetime(df['fecha_llegada'])
df['mes'] = df['fecha_llegada'].dt.month

def get_trimestre(mes):
    return (mes - 1) // 3 + 1

df['trimestre'] = df['mes'].apply(get_trimestre)

def asignar_grupo_edad(row):
    if row['edad_18-30'] == 1:
        return '18-30'
    elif row['edad_31-50'] == 1:
        return '31-50'
    elif row['edad_51-65'] == 1:
        return '51-65'
    elif row['edad_65+'] == 1:
        return '65+'
    else:
        return 'desconocido'

df['grupo_edad'] = df.apply(asignar_grupo_edad, axis=1)

# Variable de cantidad de reservas
df['cantidad_reservas'] = 1

# Columnas para agrupación
group_cols = ['hotel', 'año', 'trimestre', 'tiene_campaña']

agg_df = df.groupby(group_cols).agg(
    cantidad_reservas=('cantidad_reservas', 'sum'),
    review_sentiment_promedio=('review_sentiment_bin', 'mean'),
    cliente_recurrente_rate=('cliente_recurrente', 'mean'),
    cancelado_rate=('cancelado', 'mean'),
    promedio_adultos=('adultos', 'mean'),
    promedio_ninos=('ninos', 'mean')
).reset_index()

# Agregar columnas categóricas por moda
def modo_agg(series):
    return series.mode().iloc[0] if not series.mode().empty else np.nan

cat_cols = ['publico_objetivo', 'grupo_edad']
for col in cat_cols:
    moda = df.groupby(group_cols)[col].agg(modo_agg).reset_index()
    agg_df = agg_df.merge(moda, on=group_cols, how='left')

In [15]:
# -----------------------------
# 3. SEPARAR MODELOS CON/SIN CAMPAÑA
# -----------------------------
df_con = agg_df[agg_df['tiene_campaña'] == 1].copy()
df_sin = agg_df[agg_df['tiene_campaña'] == 0].copy()

y_con = df_con['cantidad_reservas']
X_con = df_con.drop(columns=['cantidad_reservas'])

y_sin = df_sin['cantidad_reservas']
X_sin = df_sin.drop(columns=['cantidad_reservas'])

In [16]:
# -----------------------------
# 4. PIPELINE Y ENTRENAMIENTO 
# -----------------------------
cat_features = ['hotel', 'publico_objetivo', 'grupo_edad']
num_features = [col for col in X_con.columns if col not in cat_features + ['año', 'trimestre', 'tiene_campaña']]

# Paso 1 (Nuevo): Recopilar todas las categorías únicas para cada característica categórica
# Esto garantiza que el OneHotEncoder "conozca" todas las posibilidades.
categorical_features_with_all_categories = {}
for col in cat_features:
    categorical_features_with_all_categories[col] = list(agg_df[col].unique())

# Crear el OneHotEncoder con categorías explícitas y handle_unknown='ignore'
# Usamos OneHotEncoder(categories=...) para forzar al encoder a usar un conjunto fijo de categorías.
# Si una categoría no está en este conjunto, será ignorada.
# Si una categoría esperada no está en el 'fit' de los datos, igualmente la columna existirá (será todo ceros).
# Esta es la clave para la consistencia.
ohe_transformer = OneHotEncoder(categories=[categorical_features_with_all_categories[col] for col in cat_features],
                                handle_unknown='ignore')


preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', ohe_transformer, cat_features) # Usar el ohe_transformer pre-configurado
])

# No necesitamos hacer un 'preprocessor.fit(X_total_for_preprocessor_fit)'
# porque ya hemos especificado las categorías al crear ohe_transformer.
# El 'fit' del pipeline lo hará automáticamente sobre las X de cada modelo.


# Modelo con campaña
reg_con = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(random_state=42))
])
reg_con.fit(X_con, y_con)

# Modelo sin campaña
reg_sin = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(random_state=42))
])
reg_sin.fit(X_sin, y_sin)


# Mostrar columnas transformadas y usadas por el modelo
# Obtener las categorías del OHE después de que el pipeline ha sido ajustado
# o directamente de la configuración que le dimos.
ohe_feature_names = reg_con.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(cat_features)


num_feature_names = num_features

all_features = np.concatenate([num_feature_names, ohe_feature_names])
print("_"*50)
print("Total columnas usadas por el modelo:", len(all_features))
print("\nColumnas transformadas y usadas por el modelo:")
print(all_features)

# Para verificar el número de características que el preprocesador genera en X_con y X_sin:
X_con_transformed_shape = preprocessor.transform(X_con).shape[1]
X_sin_transformed_shape = preprocessor.transform(X_sin).shape[1]

print(f"Número de características después del preprocesamiento para X_con: {X_con_transformed_shape}")
print(f"Número de características después del preprocesamiento para X_sin: {X_sin_transformed_shape}")

print("X_con shape (antes del preprocesamiento):", X_con.shape)
print("Columns usadas en X_con (antes del preprocesamiento):", X_con.columns.tolist())
print("_"*50)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 50, number of used features: 6
[LightGBM] [Info] Start training from score 1001.280000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [17]:

# -----------------------------
# 5. EVALUACIÓN EN TRAINING
# -----------------------------
print("\n--- Evaluación en entrenamiento ---")

# Predicción en el mismo conjunto usado para entrenamiento (training set)
pred_con = reg_con.predict(X_con)
pred_sin = reg_sin.predict(X_sin)

# Evaluación de errores en training
rmse_con = np.sqrt(mean_squared_error(y_con, pred_con))
rmse_sin = np.sqrt(mean_squared_error(y_sin, pred_sin))

print(f"RMSE modelo CON campaña: {rmse_con:.2f}")
print(f"RMSE modelo SIN campaña: {rmse_sin:.2f}")



--- Evaluación en entrenamiento ---
RMSE modelo CON campaña: 776.29
RMSE modelo SIN campaña: 63.63


c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [21]:
# -----------------------------
# 6. PREDICCIÓN NUEVO ESCENARIO
# -----------------------------

# --- Definiciones de tus diccionarios para selección ---
hoteles_idx = {
    1: 'gran hotel bali',
    2: 'mandarin oriental ritz',
    3: 'parador de cádiz',
    4: 'eurostars hotel de la reconquista',
    5: 'hotel arts barcelona'
}

publico_objetivo_idx = {
    1: 'familias',
    2: 'parejas',
    3: 'jovenes',
    4: 'tercera edad'
}


# --- Selecciones para el nuevo escenario ---
hotel_key = 4
publico_objetivo_key = 4
anio_prediccion = 2025
trimestre_prediccion = 4


nuevo_df = pd.DataFrame({
    'hotel': [hoteles_idx[hotel_key]],
    'año': [anio_prediccion],
    'trimestre': [trimestre_prediccion], # Usamos la variable de selección de trimestre
    'review_sentiment_promedio': [0.75],
    'cliente_recurrente_rate': [0.5],
    'cancelado_rate': [0.05],
    'promedio_adultos': [2],
    'promedio_ninos': [0],
    'publico_objetivo': [publico_objetivo_idx[publico_objetivo_key]],
    'grupo_edad': ['18-30'],
})

# Predecir usando el pipeline
reservas_con = reg_con.predict(nuevo_df)[0]
reservas_sin = reg_sin.predict(nuevo_df)[0]
incremento_abs = reservas_con - reservas_sin
incremento_rel = (incremento_abs / reservas_sin) - 1 if reservas_sin > 0 else 0
print("\n" + "="*70)
print("             ANALISIS DE IMPACTO DE CAMPAÑA")
print("="*70)

print("\n--- ESCENARIO DE PREDICCION ---")
print(f" Hotel Seleccionado: {hoteles_idx[hotel_key].title()}")
print(f" Periodo Analizado: Ano {anio_prediccion}, Trimestre {trimestre_prediccion}")


print(f" Publico Objetivo: {nuevo_df['publico_objetivo'].iloc[0].title()}")
print(f" Grupo de Edad: {nuevo_df['grupo_edad'].iloc[0]}")


print("\n--- RESULTADOS DE RESERVAS ESTIMADAS ---")
print(f" Sin Campana: Se estiman {reservas_sin:.0f} reservas.")
print(f" Con Campana: Se estiman {reservas_con:.0f} reservas.")

print("\n--- IMPACTO ESTIMADO DE LA CAMPANA ---")
if incremento_abs > 0:
    print(f" La campana podria generar {incremento_abs:.0f} reservas adicionales.")
    print(f" Esto representa un incremento del {incremento_rel:.2%}.")
elif incremento_abs < 0:
    print(f" Atencion: La campana podria resultar en {abs(incremento_abs):.0f} reservas menos.")
    print(f" Esto representa una disminucion del {abs(incremento_rel):.2%}.")
else:
    print(" La campana no muestra un cambio significativo en las reservas.")

print("\n" + "="*70)
print("        Optimiza tus estrategias de marketing.")
print("="*70 + "\n")


             ANALISIS DE IMPACTO DE CAMPAÑA

--- ESCENARIO DE PREDICCION ---
 Hotel Seleccionado: Eurostars Hotel De La Reconquista
 Periodo Analizado: Ano 2025, Trimestre 4
 Publico Objetivo: Tercera Edad
 Grupo de Edad: 18-30

--- RESULTADOS DE RESERVAS ESTIMADAS ---
 Sin Campana: Se estiman 762 reservas.
 Con Campana: Se estiman 1663 reservas.

--- IMPACTO ESTIMADO DE LA CAMPANA ---
 La campana podria generar 901 reservas adicionales.
 Esto representa un incremento del 18.25%.

        Optimiza tus estrategias de marketing.



c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [19]:
# Define your prediction scenario parameters (these will be constant for all hotels in this run)
anio_prediccion = 2025
trimestre_prediccion = 3 # You can change this to iterate through quarters if needed

# Other fixed scenario values for the input DataFrame
publico_objetivo_prediccion = 'joven'
grupo_edad_prediccion = '18-30'
review_sentiment_promedio_prediccion = 0.75
cliente_recurrente_rate_prediccion = 0.5
cancelado_rate_prediccion = 0.05
promedio_adultos_prediccion = 2
promedio_ninos_prediccion = 0

# List to store prediction results for each hotel
all_predictions_data = []

# Loop through each hotel in your dictionary
for hotel_key, hotel_name in hoteles_idx.items():
    # Create the input DataFrame for the current hotel and scenario
    nuevo_df = pd.DataFrame({
        'hotel': [hotel_name],
        'año': [anio_prediccion],
        'trimestre': [trimestre_prediccion],
        'review_sentiment_promedio': [review_sentiment_promedio_prediccion],
        'cliente_recurrente_rate': [cliente_recurrente_rate_prediccion],
        'cancelado_rate': [cancelado_rate_prediccion],
        'promedio_adultos': [promedio_adultos_prediccion],
        'promedio_ninos': [promedio_ninos_prediccion],
        'publico_objetivo': [publico_objetivo_prediccion],
        'grupo_edad': [grupo_edad_prediccion],
    })

    # Make predictions for the current hotel
    reservas_con = reg_con.predict(nuevo_df)[0]
    reservas_sin = reg_sin.predict(nuevo_df)[0]

    # Calculate increment metrics
    incremento_abs = reservas_con - reservas_sin
    incremento_rel = (reservas_con / reservas_sin) - 1 if reservas_sin > 0 else 0

    # Store the results for the current hotel
    all_predictions_data.append({
        'Hotel': hotel_name,
        'Año': anio_prediccion,
        'Trimestre': trimestre_prediccion,
        'Reservas SIN Campaña': round(reservas_sin),
        'Reservas CON Campaña': round(reservas_con),
        'Incremento Absoluto': round(incremento_abs),
        'Incremento Relativo (%)': f"{incremento_rel:.2%}" # Store as formatted string
    })

# Convert the list of dictionaries to a Pandas DataFrame
predictions_df = pd.DataFrame(all_predictions_data)

# Display the DataFrame (optional)
print("\n--- RESUMEN DE PREDICCIONES POR HOTEL ---")
print(predictions_df)

# Save the DataFrame to a CSV file
csv_filename = f"predicciones_campana_anio_{anio_prediccion}_T{trimestre_prediccion}.csv"
predictions_df.to_csv(csv_filename, index=False)

print(f"\nResultados guardados exitosamente en '{csv_filename}'")


--- RESUMEN DE PREDICCIONES POR HOTEL ---
                               Hotel   Año  Trimestre  Reservas SIN Campaña  \
0                    gran hotel bali  2025          3                   448   
1             mandarin oriental ritz  2025          3                  1264   
2                   parador de cádiz  2025          3                   196   
3  eurostars hotel de la reconquista  2025          3                   762   
4               hotel arts barcelona  2025          3                   196   

   Reservas CON Campaña  Incremento Absoluto Incremento Relativo (%)  
0                  1663                 1215                 271.45%  
1                  1663                  399                  31.53%  
2                  1663                 1466                 747.13%  
3                  1663                  901                 118.25%  
4                  1663                 1466                 747.13%  

Resultados guardados exitosamente en 'predicciones_camp

c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Alumno_AI\Documents\LEO\tfm_final\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid 

In [20]:
# # -----------------------------
# # 7. GUARDAR MODELOS
# # -----------------------------

# import joblib
# # Guardar los modelos entrenados
# joblib.dump(reg_con, '../models/regressor_con_campana.pkl')
# joblib.dump(reg_sin, '../models/regressor_sin_campana.pkl')
# # Guardar el preprocesador
# joblib.dump(preprocessor, '../models/preprocessor.pkl')